# * VINSIGHT : Churn Subs
    Voluntary Churn Subs : 
        Include -> Closing or Switch off, Postpaid to Prepaid, MNP Port-out
        Exclude -> Change owner(เปลี่ยนเบอร์), Post to Pre, Open&Close within same day
    Churn Values : บ้าน T, D ยังมี definition ต่างกันอยู่(DTAC ถูกต้องมากกว่า, TRUE น่าจะแก้ตามทีหลัง)
    Churn Rate : คือ Churn Subs / average(Reported Subbase end of last 2 month)

    * Unbalance Adjustment : เกิดมาจากเคส Open&Close within same day

    * Prepaid Churn Subs : TMH ยังใช้ไม่ได้ รอ verify ตัวเลข

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000200' --Prepaid Churn Subs
        , 'TB1S000200' --Prepaid Churn Subs : TMH
        , 'DB1S000200' --Prepaid Churn Subs : DTAC
        
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'TB2S010200' --Postpaid Churn Subs B2C : TMH
        , 'DB2S010200' --Postpaid Churn Subs B2C : DTAC
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'TB2S010201' --Postpaid Churn Subs Voluntary B2C : TMH
        , 'DB2S010201' --Postpaid Churn Subs Voluntary B2C : DTAC
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TB2S010202' --Postpaid Churn Subs Involuntary B2C : TMH
        , 'DB2S010202' --Postpaid Churn Subs Involuntary B2C : DTAC
        , 'TB2S010210' --Postpaid Churn Subs B2C : TMH (Most Used Location)
        , 'TB2S010211' --Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
        , 'TB2S010212' --Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
        
        , 'B2S020200' --Postpaid Churn Subs B2B
        , 'TB2S020200' --Postpaid Churn Subs B2B : TMH
        , 'DB2S020200' --Postpaid Churn Subs B2B : DTAC
        , 'B2S020201' --Postpaid Churn Subs Voluntary B2B
        , 'TB2S020201' --Postpaid Churn Subs Voluntary B2B : TMH
        , 'DB2S020201' --Postpaid Churn Subs Voluntary B2B : DTAC
        , 'B2S020202' --Postpaid Churn Subs Involuntary B2B
        , 'TB2S020202' --Postpaid Churn Subs Involuntary B2B : TMH
        , 'DB2S020202' --Postpaid Churn Subs Involuntary B2B : DTAC
        , 'TB2S020210' --Postpaid Churn Subs B2B : TMH (Most Used Location)
        , 'TB2S020211' --Postpaid Churn Subs Voluntary B2B : TMH (Most Used Location)
        , 'TB2S020212' --Postpaid Churn Subs Involuntary B2B : TMH (Most Used Location)
        
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-10-27, 12:53:26

DataFrame: 21512 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview by Period

In [4]:
''' Churn Subs Yearly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
churn_sub_yearly_df = churn_sub_yearly_df.loc[churn_sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_yearly_df['PRE'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B1S000200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['PRE_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB1S000200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['PRE_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB1S000200', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='B2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL_T'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TB2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['POST_B2C_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010200', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_VL_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010201', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['POST_B2C_IVL_D'] = np.where(churn_sub_yearly_df['METRIC_CD']=='DB2S010202', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['FTTX'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13100', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13102', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13103', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['FTTX_IVL(CF)'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER13104', churn_sub_yearly_df['P'], 0)

churn_sub_yearly_df['TVS'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14100', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['TVS_VL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14102', churn_sub_yearly_df['P'], 0)
churn_sub_yearly_df['TVS_IVL'] = np.where(churn_sub_yearly_df['METRIC_CD']=='TSER14103', churn_sub_yearly_df['P'], 0)


# ''' Full '''
# churn_sub_yearly_df = churn_sub_yearly_df.groupby('TM_KEY_YR')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_yearly_df = churn_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# churn_sub_yearly_df = churn_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_yearly_df_display = churn_sub_yearly_df.copy()
# churn_sub_yearly_df_display['ACTUAL_AS_OF'] = churn_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_yearly_df_display[col] = churn_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_yearly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_yearly_df = churn_sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_yearly_df = churn_sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
churn_sub_yearly_df = churn_sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_yearly_df_display = churn_sub_yearly_df.copy()
churn_sub_yearly_df_display['ACTUAL_AS_OF'] = churn_sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_yearly_df_display[col] = churn_sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,2024,2025-09-22 06:15:44,20241231,0,"2,857,216","1,066,524","1,862,336","456,668","170,154","281,222","194,250","84,176","18,800"
1,2025,2025-10-27 06:22:35,20251025,"2,743,203","1,860,186","748,743","1,137,029","348,675","136,332","240,179","181,730","18,237",135


In [5]:
''' Churn Subs Quarterly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
churn_sub_quarterly_df = churn_sub_quarterly_df.loc[churn_sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_quarterly_df['PRE'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B1S000200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['PRE_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB1S000200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['PRE_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB1S000200', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='B2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL_T'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TB2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['POST_B2C_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010200', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_VL_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010201', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['POST_B2C_IVL_D'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='DB2S010202', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['FTTX'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13100', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13102', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13103', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['FTTX_IVL(CF)'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER13104', churn_sub_quarterly_df['P'], 0)

churn_sub_quarterly_df['TVS'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14100', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['TVS_VL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14102', churn_sub_quarterly_df['P'], 0)
churn_sub_quarterly_df['TVS_IVL'] = np.where(churn_sub_quarterly_df['METRIC_CD']=='TSER14103', churn_sub_quarterly_df['P'], 0)


# ''' Full '''
# churn_sub_quarterly_df = churn_sub_quarterly_df.groupby('TM_KEY_QTR')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_quarterly_df = churn_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# churn_sub_quarterly_df = churn_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_quarterly_df_display = churn_sub_quarterly_df.copy()
# churn_sub_quarterly_df_display['ACTUAL_AS_OF'] = churn_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_quarterly_df_display[col] = churn_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_quarterly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_quarterly_df = churn_sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_quarterly_df = churn_sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
churn_sub_quarterly_df = churn_sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_quarterly_df_display = churn_sub_quarterly_df.copy()
churn_sub_quarterly_df_display['ACTUAL_AS_OF'] = churn_sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_quarterly_df_display[col] = churn_sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,20241,2025-09-22 06:15:44,20240331,0,"871,395","318,353","576,878","118,742","43,223","72,945","48,913","37,318","13,347"
1,20242,2025-09-22 06:15:44,20240630,0,"718,051","262,796","454,251","114,261","43,677","70,807","40,970","35,580","5,046"
2,20243,2025-09-22 06:15:44,20240930,0,"664,232","248,339","433,905","118,872","44,269","67,909","48,795","5,040",355
3,20244,2025-09-22 06:15:44,20241231,0,"603,538","237,036","397,302","104,793","38,985","69,561","55,572","6,238",52
4,20251,2025-10-27 06:22:35,20250331,"7,728,433","553,494","225,501","341,127","104,815","38,083","68,600","47,799","5,854",35
5,20252,2025-10-27 06:22:35,20250630,"6,849,967","586,573","243,446","339,713","92,588","43,483","70,494","66,735","5,905",44
6,20253,2025-10-27 06:22:35,20250930,"-13,175,131","590,961","225,842","373,167","121,642","44,568","80,563","67,196","6,478",56
7,20254,2025-10-27 06:22:35,20251025,"1,339,934","129,158","53,954","83,022","29,630","10,198","20,522",0,0,0


In [6]:
''' Churn Subs Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
churn_sub_monthly_df = churn_sub_monthly_df.loc[churn_sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_monthly_df['PRE'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B1S000200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['PRE_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB1S000200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['PRE_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB1S000200', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='B2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL_T'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TB2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['POST_B2C_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010200', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_VL_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010201', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['POST_B2C_IVL_D'] = np.where(churn_sub_monthly_df['METRIC_CD']=='DB2S010202', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['FTTX'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13100', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13102', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13103', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['FTTX_IVL(CF)'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER13104', churn_sub_monthly_df['P'], 0)

churn_sub_monthly_df['TVS'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14100', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['TVS_VL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14102', churn_sub_monthly_df['P'], 0)
churn_sub_monthly_df['TVS_IVL'] = np.where(churn_sub_monthly_df['METRIC_CD']=='TSER14103', churn_sub_monthly_df['P'], 0)


# ''' Full '''
# churn_sub_monthly_df = churn_sub_monthly_df.groupby('TM_KEY_MTH')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_monthly_df = churn_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# churn_sub_monthly_df = churn_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_monthly_df_display = churn_sub_monthly_df.copy()
# churn_sub_monthly_df_display['ACTUAL_AS_OF'] = churn_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_monthly_df_display[col] = churn_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_monthly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_monthly_df = churn_sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_monthly_df = churn_sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
churn_sub_monthly_df = churn_sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_monthly_df_display = churn_sub_monthly_df.copy()
churn_sub_monthly_df_display['ACTUAL_AS_OF'] = churn_sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_monthly_df_display[col] = churn_sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,202401,2025-09-22 06:15:44,20240131,0,"335,195","118,348","231,754","40,468","16,454","24,014","14,004","10,196","3,820"
1,202402,2025-09-22 06:15:44,20240229,0,"281,024","102,361","180,543","38,531","14,057","24,474","12,774","7,689","6,795"
2,202403,2025-09-22 06:15:44,20240331,0,"255,176","97,644","164,581","39,743","12,712","24,457","22,135","19,433","2,732"
3,202404,2025-09-22 06:15:44,20240430,0,"247,654","90,553","155,090","38,455","13,522","24,520","20,819","18,602","2,217"
4,202405,2025-09-22 06:15:44,20240531,0,"250,447","92,059","151,641","38,397","14,955","23,442","16,541","14,731","1,810"
5,202406,2025-09-22 06:15:44,20240630,0,"219,950","80,184","147,520","37,409","15,200","22,845","3,610","2,247","1,019"
6,202407,2025-09-22 06:15:44,20240731,0,"232,711","91,260","145,540","40,306","15,934","22,944","9,003","1,136",22
7,202408,2025-09-22 06:15:44,20240831,0,"224,948","81,816","143,964","40,742","14,958","22,914","21,101","2,225",313
8,202409,2025-09-22 06:15:44,20240930,0,"206,573","75,263","144,401","37,824","13,377","22,051","18,691","1,679",20
9,202410,2025-09-22 06:15:44,20241031,0,"207,481","83,683","135,820","38,097","13,580","23,214","19,574","2,181",30


In [7]:
''' Churn Subs Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_sub_weekly_df = churn_sub_weekly_df.loc[churn_sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_weekly_df['PRE'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B1S000200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['PRE_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB1S000200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['PRE_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB1S000200', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='B2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL_T'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TB2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['POST_B2C_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010200', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_VL_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010201', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['POST_B2C_IVL_D'] = np.where(churn_sub_weekly_df['METRIC_CD']=='DB2S010202', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['FTTX'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13100', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13102', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13103', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['FTTX_IVL(CF)'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER13104', churn_sub_weekly_df['P'], 0)

churn_sub_weekly_df['TVS'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14100', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['TVS_VL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14102', churn_sub_weekly_df['P'], 0)
churn_sub_weekly_df['TVS_IVL'] = np.where(churn_sub_weekly_df['METRIC_CD']=='TSER14103', churn_sub_weekly_df['P'], 0)


# ''' Full '''
# churn_sub_weekly_df = churn_sub_weekly_df.groupby('TM_KEY_WK')\
#     .agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_weekly_df = churn_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# churn_sub_weekly_df = churn_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_weekly_df_display = churn_sub_weekly_df.copy()
# churn_sub_weekly_df_display['ACTUAL_AS_OF'] = churn_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
# mod_col_list = churn_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_weekly_df_display[col] = churn_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_weekly_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_weekly_df = churn_sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_weekly_df = churn_sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
churn_sub_weekly_df = churn_sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_weekly_df_display = churn_sub_weekly_df.copy()
churn_sub_weekly_df_display['ACTUAL_AS_OF'] = churn_sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = churn_sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_weekly_df_display[col] = churn_sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,2025036,2025-10-27 06:22:35,20250907,"353,171","48,163","17,422","29,496","11,089","3,425","3,514","5,268",347,0
1,2025037,2025-10-27 06:22:35,20250914,"390,712","45,322","15,117","27,164","11,995","2,942","8,068","4,849",310,0
2,2025038,2025-10-27 06:22:35,20250921,"377,916","35,527","14,329","23,590","7,877","2,753","6,298","4,222",300,1
3,2025039,2025-10-27 06:22:35,20250928,"373,294","46,497","14,785","32,490","9,228","2,808","10,426","3,216",266,1
4,2025040,2025-10-27 06:22:35,20251005,"330,180","32,632","17,085","21,135","3,989","3,343","6,646",0,0,0
5,2025041,2025-10-27 06:22:35,20251012,"349,792","40,561","16,085","25,030","11,483","3,178","4,310",0,0,0
6,2025042,2025-10-27 06:22:35,20251019,"387,572","46,229","15,506","28,106","8,030","2,982","5,828",0,0,0
7,2025043,2025-10-27 06:22:35,20251025,"377,192","18,943","11,082","16,769","3,536","1,614","3,749",0,0,0


In [8]:
''' Churn Subs Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'TB1S000200' #Prepaid Churn Subs : TMH
    , 'DB1S000200' #Prepaid Churn Subs : DTAC
    , 'B2S010200' #Postpaid Churn Subs B2C
    , 'TB2S010200' #Postpaid Churn Subs B2C : TMH
    , 'DB2S010200' #Postpaid Churn Subs B2C : DTAC
    , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    , 'TB2S010201' #Postpaid Churn Subs Voluntary B2C : TMH
    , 'DB2S010201' #Postpaid Churn Subs Voluntary B2C : DTAC
    , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TB2S010202' #Postpaid Churn Subs Involuntary B2C : TMH
    , 'DB2S010202' #Postpaid Churn Subs Involuntary B2C : DTAC
    # , 'TB2S010210' #Postpaid Churn Subs B2C : TMH (Most Used Location)
    # , 'TB2S010211' #Postpaid Churn Subs Voluntary B2C : TMH (Most Used Location)
    # , 'TB2S010212' #Postpaid Churn Subs Involuntary B2C : TMH (Most Used Location)
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    , 'TSER14102' #TVS Churn Subs Voluntary
    , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_sub_daily_df = churn_sub_daily_df.loc[churn_sub_daily_df['METRIC_CD'].isin(v_metric_list)]

churn_sub_daily_df['PRE'] = np.where(churn_sub_daily_df['METRIC_CD']=='B1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB1S000200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['PRE_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB1S000200', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='B2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_T'] = np.where(churn_sub_daily_df['METRIC_CD']=='TB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['POST_B2C_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010200', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_VL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010201', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['POST_B2C_IVL_D'] = np.where(churn_sub_daily_df['METRIC_CD']=='DB2S010202', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['FTTX'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13103', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['FTTX_IVL(CF)'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER13104', churn_sub_daily_df['P'], 0)

churn_sub_daily_df['TVS'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14100', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_VL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14102', churn_sub_daily_df['P'], 0)
churn_sub_daily_df['TVS_IVL'] = np.where(churn_sub_daily_df['METRIC_CD']=='TSER14103', churn_sub_daily_df['P'], 0)


# ''' Full '''
# churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY'])\
#     .agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum'
#           , 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum'
#           , 'POST_B2C_T':'sum', 'POST_B2C_VL_T':'sum', 'POST_B2C_IVL_T':'sum'
#           , 'POST_B2C_D':'sum', 'POST_B2C_VL_D':'sum', 'POST_B2C_IVL_D':'sum'
#           , 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum'
#           , 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
# churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM'
#                                          , 'PRE', 'PRE_T', 'PRE_D'
#                                          , 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL'
#                                          , 'POST_B2C_T', 'POST_B2C_VL_T', 'POST_B2C_IVL_T'
#                                          , 'POST_B2C_D', 'POST_B2C_VL_D', 'POST_B2C_IVL_D'
#                                          , 'FTTX', 'FTTX_VL', 'FTTX_IVL'
#                                          , 'TVS', 'TVS_VL', 'TVS_IVL']]
# churn_sub_daily_df_display = churn_sub_daily_df.copy()
# mod_col_list = churn_sub_daily_df_display.iloc[:, 3:].columns.tolist()
# for col in mod_col_list:
#     churn_sub_daily_df_display[col] = churn_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
# print(f'\n{churn_sub_daily_df_display.to_string(max_cols=100)}') #max_rows=1000


''' Overview '''
churn_sub_daily_df = churn_sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'POST_B2C':'sum', 'POST_B2C_VL':'sum', 'POST_B2C_IVL':'sum', 'FTTX':'sum', 'FTTX_VL':'sum', 'FTTX_IVL':'sum', 'TVS':'sum', 'TVS_VL':'sum', 'TVS_IVL':'sum'})
churn_sub_daily_df = churn_sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_sub_daily_df = churn_sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'POST_B2C', 'POST_B2C_VL', 'POST_B2C_IVL', 'FTTX', 'FTTX_VL', 'FTTX_IVL', 'TVS', 'TVS_VL', 'TVS_IVL']]
churn_sub_daily_df_display = churn_sub_daily_df.copy()
mod_col_list = churn_sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_sub_daily_df_display[col] = churn_sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
churn_sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,POST_B2C,POST_B2C_VL,POST_B2C_IVL,FTTX,FTTX_VL,FTTX_IVL,TVS,TVS_VL,TVS_IVL
0,202510,20251026,2025-10-27 06:22:35,0,0,0,0,0,0,0,0,0,0
1,202510,20251025,2025-10-27 06:22:35,"68,870",0,0,0,0,0,0,0,0,0
2,202510,20251024,2025-10-27 06:22:35,"63,901","3,322","1,991","2,593",0,0,0,0,0,0
3,202510,20251023,2025-10-27 06:22:35,"59,082","3,850","1,363","3,090",-93,428,0,0,0,0
4,202510,20251022,2025-10-27 06:22:35,"65,121","3,174","1,974","5,553",-370,360,"3,749",0,0,0
5,202510,20251021,2025-10-27 06:22:35,"60,190","5,136","3,709","2,313","4,353",462,0,0,0,0
6,202510,20251020,2025-10-27 06:22:35,"60,028","3,461","2,045","3,220",-354,364,0,0,0,0
7,202510,20251019,2025-10-27 06:22:35,"61,645","2,867","1,756","2,053",-280,412,0,0,0,0
8,202510,20251018,2025-10-27 06:22:35,"50,040","13,208","1,606","6,920","4,668",428,0,0,0,0
9,202510,20251017,2025-10-27 06:22:35,"52,645","5,022","1,919","3,978",-304,422,"1,947",0,0,0


## Products Summary

### Prep Monthly Data

In [9]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [10]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202509


### Prepaid

In [11]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Prepaid,B1S000200,Prepaid Churn Subs,2025-10-27 06:22:35,0,"1,599,895","2,142,866","2,142,866","2,142,866","-1,599,895"
1,202509,Prepaid,DB1S000200,Prepaid Churn Subs : DTAC,2025-10-27 06:22:35,0,"758,399","758,557","758,557","758,557","-758,399"
2,202509,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"841,496","1,384,309","1,384,309","1,384,309","-841,496"


### Postpaid

In [12]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)


In [13]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Postpaid,B2S010200,Postpaid Churn Subs B2C,2025-10-27 06:22:35,"184,716","184,716","184,398","184,398","184,398",0
1,202509,Postpaid,B2S010201,Postpaid Churn Subs Voluntary B2C,2025-10-27 06:22:35,"67,457","67,457","67,145","67,145","67,146",0
2,202509,Postpaid,B2S010202,Postpaid Churn Subs Involuntary B2C,2025-10-27 06:22:35,"120,758","120,758","120,754","120,754","120,754",0
3,202509,Postpaid,DB2S010200,Postpaid Churn Subs B2C : DTAC,2025-10-27 06:22:35,"55,259","55,259","55,113","55,113","55,113",0
4,202509,Postpaid,DB2S010201,Postpaid Churn Subs Voluntary B2C : DTAC,2025-10-27 06:22:35,"19,283","19,283","19,140","19,140","19,140",0
5,202509,Postpaid,DB2S010202,Postpaid Churn Subs Involuntary B2C : DTAC,2025-10-27 06:22:35,"39,475","39,475","39,474","39,474","39,474",0
6,202509,Postpaid,TB2S010200,Postpaid Churn Subs B2C : TMH,2025-10-27 06:22:35,"129,457","129,457","129,285","129,285","129,285",0
7,202509,Postpaid,TB2S010201,Postpaid Churn Subs Voluntary B2C : TMH,2025-10-27 06:22:35,"48,174","48,174","48,005","48,005","48,006",0
8,202509,Postpaid,TB2S010202,Postpaid Churn Subs Involuntary B2C : TMH,2025-10-27 06:22:35,"81,283","81,283","81,280","81,280","81,280",0
9,202509,Postpaid,TB2S010210,Postpaid Churn Subs B2C : TMH (Most Used Locat...,2025-10-27 06:22:35,0,0,0,0,0,0


In [14]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,Postpaid,B2S020200,Postpaid Churn Subs B2B,2025-10-27 03:30:39,0,"32,005","32,004","32,001",0,"-32,005"
1,202509,Postpaid,B2S020201,Postpaid Churn Subs Voluntary B2B,2025-10-27 03:30:39,0,"29,854","29,853","29,850",0,"-29,854"
2,202509,Postpaid,B2S020202,Postpaid Churn Subs Involuntary B2B,2025-10-27 03:30:39,0,"2,151","2,151","2,151",0,"-2,151"
3,202509,Postpaid,DB2S020200,Postpaid Churn Subs B2B : DTAC,2025-10-27 03:30:39,0,"12,196","12,196","12,193",0,"-12,196"
4,202509,Postpaid,DB2S020201,Postpaid Churn Subs Voluntary B2B : DTAC,2025-10-27 03:30:39,0,"11,629","11,629","11,626",0,"-11,629"
5,202509,Postpaid,DB2S020202,Postpaid Churn Subs Involuntary B2B : DTAC,2025-10-27 03:30:39,0,567,567,567,0,-567
6,202509,Postpaid,TB2S020200,Postpaid Churn Subs B2B : TMH,2025-10-27 06:22:35,0,"19,809","19,808","19,808","19,808","-19,809"
7,202509,Postpaid,TB2S020201,Postpaid Churn Subs Voluntary B2B : TMH,2025-10-27 06:22:35,0,"18,225","18,224","18,224","18,224","-18,225"
8,202509,Postpaid,TB2S020202,Postpaid Churn Subs Involuntary B2B : TMH,2025-10-27 06:22:35,0,"1,584","1,584","1,584","1,584","-1,584"
9,202509,Postpaid,TB2S020210,Postpaid Churn Subs B2B : TMH (Most Used Locat...,2025-10-27 06:22:35,0,"19,809","19,808","19,808","19,808","-19,809"


### TOL

In [15]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,TOL,TSER13100,FTTx Churn Subs,2025-10-27 06:22:35,0,"37,597","37,593","37,593","37,593","-37,597"
1,202509,TOL,TSER13102,FTTx Churn Subs Voluntary,2025-10-27 06:22:35,0,"12,847","12,843","12,843","12,843","-12,847"
2,202509,TOL,TSER13103,FTTx Churn Subs Involuntary,2025-10-27 06:22:35,0,"28,317","28,317","28,317","28,317","-28,317"
3,202509,TOL,TSER13104,FTTx Churn Subs Involuntary (Collection Forecast),2025-10-27 06:22:35,0,"28,882","28,882","28,882","28,882","-28,882"


### TVS

In [16]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202509,TVS,TSER14100,TVS Churn Subs,2025-10-27 03:30:39,0,"17,555","17,555","16,984",0,"-17,555"
1,202509,TVS,TSER14102,TVS Churn Subs Voluntary,2025-10-27 03:30:39,0,"1,223","17,553","16,984",0,"-1,223"
2,202509,TVS,TSER14103,TVS Churn Subs Involuntary,2025-10-27 03:30:39,0,2,2,1,0,-2


## ** Current Issue

In [17]:
''' Monthly : Prepaid Churn Subs : TMH (Align with BU) '''

v_metric_cd = 'TB1S000200'

issue_prepaid_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.loc[issue_prepaid_tmh_df['TM_KEY_MTH']>=202401]
issue_prepaid_tmh_df = issue_prepaid_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_prepaid_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202501,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"1,004,368","637,257","637,257","638,760","-1,004,368"
1,202502,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"1,221,001","939,373","939,373","939,373","-1,221,001"
2,202503,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"1,511,810","1,277,467","1,277,467","1,277,467","-1,511,810"
3,202504,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"1,287,781","1,066,130","1,066,130","1,066,130","-1,287,781"
4,202505,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"1,132,193","1,250,315","1,250,315","1,250,315","-1,132,193"
5,202506,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"1,344,826","1,772,623","1,772,623","1,772,623","-1,344,826"
6,202507,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"1,139,159","1,732,006","1,732,006","1,732,006","-1,139,159"
7,202508,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"-17,696,192","-17,208,049","-17,208,049","-17,208,049","17,696,192"
8,202509,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"841,496","1,384,309","1,384,309","1,384,309","-841,496"
9,202510,Prepaid,TB1S000200,Prepaid Churn Subs : TMH,2025-10-27 06:22:35,0,"622,836","1,138,551","1,138,551","1,138,551","-622,836"
